In [ ]:
# Cell 1: Install necessary libraries
!pip install --upgrade transformers datasets
!pip install --upgrade unsloth
!pip install --upgrade torch
!pip install nvidia-tensorrt
# You can use localtunnel as nother option to expose your end point.
# # #!npm install -g localtunnel

yes
  Using cached transformers-4.46.3-py3-none-any.whl.metadata (44 kB)
  Using cached tokenizers-0.20.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached transformers-4.46.3-py3-none-any.whl (10.0 MB)
Using cached tokenizers-0.20.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.0
    Uninstalling tokenizers-0.21.0:
      Successfully uninstalled tokenizers-0.21.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.1
    Uninstalling transformers-4.47.1:
      Successfully uninstalled transformers-4.47.1
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for tensorrt: filename=tensorrt-10.7.0-py2.py3-none-any.whl size=16336 sha256=46348a0ee4ac3f90fd62577177ca7126e2b53ed0321654dee49a291f9cb3de26
  Stored in directory: /root/.cache/pip/wheels/da/cb/16/d5add64df49

In [ ]:
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

Found existing installation: unsloth 2024.12.8
Uninstalling unsloth-2024.12.8:
  Successfully uninstalled unsloth-2024.12.8
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-req-build-52z184xb
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-req-build-52z184xb
  Resolved https://github.com/unslothai/unsloth.git to commit 79db7130734056928fc25e38a0fde64b72d5615e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2024.12.8-py3-none-any.whl size=174222 sha256=d30c54c0b5c20451be3ff3ec7bb81e2b336115bb9086352b88bf00251b42e8d1
  Stored in directory: /tmp/pip-ephem-wheel-cache-0u4ajqfh/wheels/ed/d4/e9/76fb290ee3df0a5fc21ce5c2c788e29e9607a2353d8342fd0d
Successfully built unsloth


In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,

)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.12.8: Fast Llama patching. Transformers: 4.46.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.12.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("code_search_net", "java", split="train")

# EOS Token
EOS_TOKEN = "<|endoftext|>"

# Function to format the dataset for model input/output
def formatting_prompts_func(examples):
    problematic_codes = examples["func_code_string"]  # Get the code from the dataset
    optimized_codes = examples["whole_func_string"]  # Use the whole function (including docstring) as the output (or provide optimized versions manually)

    texts = []
    for problematic_code, optimized_code in zip(problematic_codes, optimized_codes):
        # Create a prompt with instruction to identify issues in code and provide optimization
        text = f"### Instruction:\nIdentify potential problems in the following Java code and provide an optimized solution and state its O-complexity.\n\n### Input:\n{problematic_code}\n\n### Response:\n{optimized_code}{EOS_TOKEN}"
        texts.append(text)

    return {"text": texts}

# Apply the formatting function to the dataset
dataset = dataset.map(formatting_prompts_func, batched=True)

# Example of formatted output
print(dataset[0])  # Check the first example to ensure formatting is correct


Map:   0%|          | 0/454451 [00:00<?, ? examples/s]

{'repository_name': 'OpenLiberty/open-liberty', 'func_path_in_repository': 'dev/com.ibm.ws.wsoc/src/com/ibm/ws/wsoc/external/WebSocketContainerExt.java', 'func_name': 'WebSocketContainerExt.connectToServer', 'whole_func_string': '@Override\n    public Session connectToServer(Class<? extends Endpoint> endpointClass, ClientEndpointConfig endpointConfig, URI path) throws DeploymentException, IOException {\n\n        Object theObject = null;\n\n        if (endpointClass == null || path == null) {\n            throw new IllegalArgumentException();\n        }\n\n        if (!(Endpoint.class.isAssignableFrom(endpointClass))) {\n            DeploymentException e = new DeploymentException("Class " + endpointClass.getName() + " does not extend Endpoint");\n            String msg = Tr.formatMessage(tc, "client.invalid.endpointclass", endpointClass.toString(), e.getMessage());\n            Tr.error(tc, "client.invalid.endpointclass", endpointClass.toString(), e.getMessage());\n            throw ne

In [ ]:
print(len(dataset))


454451


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "wandb", # Use this for WandB etc
    ),
)

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
!pip install accelerator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.2/90.2 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.2/56.2 kB 3.5 MB/s eta 0:00:00


In [ ]:
training_arguments = TrainingArguments(
    output_dir="./fine_tuned_model",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    num_train_epochs=30,
    save_strategy="steps",
    save_steps=100,
    logging_steps=10,
    evaluation_strategy="steps",
    eval_steps=100,
    fp16=True
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from accelerate import Accelerator

# Initialize the Accelerator
accelerator = Accelerator()

# Ensure that your model, dataset, etc., are placed on the correct device
model, optimizer, train_dataset = accelerator.prepare(model, tokenizer,dataset)

# Start the training loop
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 454,451 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 83,886,080
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
1,1.026600
2,1.175500
3,0.884900
4,1.015700
5,0.850900
6,1.287900
7,0.760800
8,0.523200
9,0.744600
10,0.654400


TrainOutput(global_step=60, training_loss=0.6352130676309268, metrics={'train_runtime': 872.8228, 'train_samples_per_second': 0.55, 'train_steps_per_second': 0.069, 'total_flos': 1.0292977220124672e+16, 'train_loss': 0.6352130676309268, 'epoch': 0.0010562171582477358})

In [ ]:
# Grant access to hugging face
from google.colab import userdata
HF_TOKEN = userdata.get('HF_TOKEN')


In [ ]:
# Push my fine tuned model with its tokenizer to my hugging face repo
repo_path="BSAtlas/BSCode-1-Stable"
model.push_to_hub(
   repo_path,
   save_method = "merged_4bit",
   token=HF_TOKEN
)
tokenizer.push_to_hub(
  repo_path,
  save_method = "merged_4bit",
  token=HF_TOKEN
)

README.md:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/336M [00:00<?, ?B/s]

Saved model to https://huggingface.co/BSAtlas/BSCode_4bit


README.md:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]